# Load libraries and files

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd

from scipy import stats
from scipy.stats import norm, skew
from scipy.stats.stats import pearsonr

from sklearn import ensemble, tree, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display
pd.set_option('display.float_format', lambda x: '%.3f' %x)
import warnings
warnings.filterwarnings('ignore')

# Look for Files
import os
print(os.listdir("../input"))

In [ ]:
# Make dataframe for train and test sets
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
print('Train dataframe size: ', df_train.shape)
print('Test dataframe size: ', df_test.shape)

In [ ]:
# View sample of data
df_train.sample(10)

In [ ]:
# Get list of all features
features = df_train.columns.tolist()
print('Features:')
features

In [ ]:
# Describe features
df_train.describe(include='all')

In [ ]:
# Make sure there are no duplicate ids
idsUnique = len(set(df_train.Id))
idsTotal = df_train.shape[0]
idsDup = idsTotal - idsUnique
if idsDup>0: print('DUPLICATE IDS FOUND')
    
# Remove ID column from dataframe
train_IDs = df_train.Id
df_train.drop('Id', axis=1, inplace=True)
test_IDs = df_test.Id
df_test.drop('Id', axis=1, inplace=True)
features.remove('Id')

print('ID column dropped')
print('Train dataframe size: ', df_train.shape)
print('Test dataframe size: ', df_test.shape)

# Coorelate features to eachother and to sales price

In [ ]:
# Coorelation plot
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(20,9))
sns.heatmap(corrmat, square=True)

In [ ]:
# Identify which features are most coorelated to eachother (redundant features)
corr_cutoff = 0.7
top_corrmat = pd.DataFrame(columns=['Feature_Correlation','Feature_1', 'Price_Correlation_1','Feature_2', 'Price_Correlation_2'])
for feature in corrmat.columns.tolist():
    temp_list = corrmat.index[abs(corrmat[feature])>corr_cutoff].tolist()
    temp_list.remove(feature) #remove diagnol elements
    if 'SalePrice' in temp_list: temp_list.remove('SalePrice') #ignore sale price for now
    if (len(temp_list)>0) & (feature != 'SalePrice'):
        for each in temp_list:
            if each not in top_corrmat['Feature_1'].values.tolist():
                feature_corr = corrmat.loc[feature, each]
                f1_sale = corrmat.loc[feature, 'SalePrice']
                f2_sale = corrmat.loc[each, 'SalePrice']
                top_corrmat = top_corrmat.append({'Feature_1': feature, 'Price_Correlation_1': f1_sale,'Feature_2': each, 'Feature_Correlation': feature_corr, 'Price_Correlation_2':f2_sale}, ignore_index=True)

print('Top Coorelated Features (cutoff = 0.7):')
top_corrmat

In [ ]:
# For each pair remove feature less coorelated with price
features.remove('GarageYrBlt')
features.remove('1stFlrSF')
features.remove('TotRmsAbvGrd')
features.remove('GarageArea')
print('Updated Features List:')
features

In [ ]:
# Update datagframes with new feature list
df_train = df_train[features]
temp_features = [i for i in features if i != 'SalePrice']
df_test = df_test[temp_features]

# Correlation plot
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(20,9))
sns.heatmap(corrmat, square=True)

In [ ]:
# Find features most coorelated with sale price
corr_cutoff = 0.5
top_corr_features = corrmat.index[abs(corrmat['SalePrice'])>corr_cutoff]
plt.figure(figsize=(10,10))
g=sns.heatmap(df_train[top_corr_features].corr(), annot=True)

In [ ]:
# Plot for most coorelated feature
sns.boxplot(x='OverallQual', y='SalePrice', data=df_train)

In [ ]:
# Scatterplots for top features
top_corr_features.drop('SalePrice')
plt.figure(figsize=(20,20))
nrows = str(int(np.ceil(len(top_corr_features)/3)))
ncols = str(3)
i=1
for feature in top_corr_features:
    plt.subplot(nrows, ncols, i)
    plt.scatter(df_train[feature], df_train.SalePrice)
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    i += 1

In [ ]:
# Remove outliers
df_train = df_train[df_train.TotalBsmtSF < 5000]
df_train = df_train[(df_train.GrLivArea < 4000) | (df_train.SalePrice > 600000)]

In [ ]:
# Replot distributions
plt.figure(figsize=(20,20))
nrows = str(int(np.ceil(len(top_corr_features)/3)))
ncols = str(3)
i=1
for feature in top_corr_features:
    plt.subplot(nrows, ncols, i)
    plt.scatter(df_train[feature], df_train.SalePrice)
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    i += 1

# Create combined feature dataframe for data cleaning
# Divide up numerical and categorical features

In [ ]:
# Save number of entries for each dataframe
ntrain = df_train.shape[0]
ntest = df_test.shape[0]

# combine train and test data with train on top
all_data = pd.concat((df_train, df_test))

# drop SalePrice from features
all_data.drop('SalePrice', axis=1, inplace=True)

print ('Dataframe sizes:')
print('Train: ', df_train.shape)
print('Test: ', df_test.shape)
print('All Data', all_data.shape)

In [ ]:
# Separate feeatures into numerical and calegorical
cat_features = all_data.select_dtypes(include=['object']).columns.values.tolist()
num_features = all_data.select_dtypes(exclude=['object']).columns.values.tolist()
print(len(cat_features), ' Cat Features')
print(cat_features)
print(len(num_features), ' Num Features')
print(num_features)

In [ ]:
# recategorize cat features that look like num features
new_cat = ['MSSubClass', 'MoSold', 'OverallCond', 'OverallQual']
for each in new_cat:
    cat_features.append(each)
    num_features.remove(each)
print(len(cat_features), ' Cat Features')
print(cat_features)
print(len(num_features), ' Num Features')
print(num_features)

In [ ]:
# Save dataframes for each feature type
num_data = all_data[num_features]
cat_data = all_data[cat_features]

# Find Missing Values

## Fill with specific value
### NA
Alley            object

Fence            object 

FireplaceQu      object

MiscFeature      object 

PoolQC           object 


### Typ
Functional       object

### TA
KitchenQual      object

### Median of Neighborhood
LotFrontage     float64

### Mode of Neighborhood
MSZoning         object 

SaleType         object 

Utilities        object 

Electrical       object 

Exterior1st      object 

Exterior2nd      object 


### 0
MasVnrArea      float64 

### None
MasVnrType       object 

## Determine if area exists
look for cat feature other than NA or sum of num features  >0
if exists, fill with mode/median of neighborhood
else fill with None/0

### Basement
BsmtCond         object 

BsmtExposure     object 

BsmtFinType1     object

BsmtFinType2     object 

BsmtQual         object 


BsmtFinSF1      float64 

BsmtFinSF2      float64 

BsmtFullBath    float64 

BsmtHalfBath    float64 

BsmtUnfSF       float64 

TotalBsmtSF     float64 


### Garage
GarageCond       object 

GarageFinish     object 

GarageQual       object 

GarageType       object 


GarageArea      float64 

GarageCars      float64 

GarageYrBlt     float64 


In [ ]:
# Find which features have missing data
all_data_missing = all_data.isnull().sum()
all_data_missing = all_data_missing[all_data_missing>0]
all_data_missing.sort_values(ascending=False)

In [ ]:
all_data[all_data_missing.index].dtypes

In [ ]:
# Look for coorelated features to detemine importance of each feature
corr_cutoff = 0.5
for each in all_data_missing.index:
    if all_data[each].dtype =='float64':
        top_features = corrmat.index[abs(corrmat[each])>corr_cutoff].values.tolist()
        top_features.remove(each)
        for ea in top_features:
            if ea not in all_data_missing.index:
                print(each, ': ', ea, ', ', corrmat[each][ea])

In [ ]:
# Fill missing values as listed in above comment
temp_list=['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC']
temp_value='NA'
for each in temp_list:
    all_data.loc[:, each] = all_data.loc[:, each].fillna(temp_value)

temp_list=['Functional']
temp_value='Typ'
for each in temp_list:
    all_data.loc[:, each] = all_data.loc[:, each].fillna(temp_value)

temp_list=['KitchenQual']
temp_value='TA'
for each in temp_list:
    all_data.loc[:, each] = all_data.loc[:, each].fillna(temp_value)

temp_list=['MasVnrType']
temp_value='None'
for each in temp_list:
    all_data.loc[:, each] = all_data.loc[:, each].fillna(temp_value)
    
temp_list=['MasVnrArea']
temp_value=0
for each in temp_list:
    all_data.loc[:, each] = all_data.loc[:, each].fillna(temp_value)

temp_list=['LotFrontage']
for each in temp_list:
    all_data.loc[:, each] = all_data.groupby(by='Neighborhood')[each].transform(lambda x: x.fillna(x.median()))

temp_list=['MSZoning', 'SaleType', 'Utilities', 'Electrical', 'Exterior1st', 'Exterior2nd']
for each in temp_list:
    all_data.loc[:, each] = all_data.groupby(by='Neighborhood')[each].transform(lambda x: x.fillna(x.mode()[0]))

In [ ]:
# Basement Features
cat_Bsmt = [i for i in cat_features if 'Bsmt' in i]
num_Bsmt = [i for i in num_features if 'Bsmt' in i]
all_Bsmt = cat_Bsmt + num_Bsmt
temp_array = []
for index, row in all_data[all_data[all_Bsmt].isnull().any(axis=1)].iterrows():
    include=False
    for each in num_Bsmt:
        if row[each]>0: include=True
    for each in cat_Bsmt:
        if type(row[each])==str:
            if row[each] != 'NA': include=True
    if include: 
        temp_array.append(index)
        for each in num_Bsmt:
            all_data.loc[index, each] = all_data.groupby(by='Neighborhood')[each].transform(lambda x: x.fillna(x.median())).iloc[index]
        for each in cat_Bsmt:
            all_data.loc[index, each] = all_data.groupby(by='Neighborhood')[each].transform(lambda x: x.fillna(x.mode()[0])).iloc[index]
    else:
        for each in num_Bsmt:
            all_data.loc[index, each] = 0
        for each in cat_Bsmt:
            all_data.loc[index, each] = 'NA'

print('Remaining missing data for Basement:')
all_data[all_Bsmt].isnull().sum()

In [ ]:
# Garage Features
cat_Gar = [i for i in cat_features if 'Garage' in i]
num_Gar = [i for i in num_features if 'Garage' in i]
all_Gar = cat_Gar + num_Gar
temp_array = []
for index, row in all_data[all_data[all_Gar].isnull().any(axis=1)].iterrows():
    include=False
    for each in num_Gar:
        if row[each]>0: include=True
    for each in cat_Gar:
        if type(row[each])==str:
            if row[each] != 'NA': include=True
    if include: 
        temp_array.append(index)
        for each in num_Gar:
            all_data.loc[index, each] = all_data.groupby(by='Neighborhood')[each].transform(lambda x: x.fillna(x.median())).iloc[index]
        for each in cat_Gar:
            all_data.loc[index, each] = all_data.groupby(by='Neighborhood')[each].transform(lambda x: x.fillna(x.mode()[0])).iloc[index]
    else:
        for each in num_Gar:
            all_data.loc[index, each] = 0
        for each in cat_Gar:
            all_data.loc[index, each] = 'NA'

print('Remaining missing data for Garage:')
all_data[all_Gar].isnull().sum()

In [ ]:
num_data = all_data[num_features]
cat_data = all_data[cat_features]
print('Number of Missing Values')
np.max(all_data.isnull().sum())

# Look at Sales Price (Target Value)
Plot histogram of sales prices and compare to normal distribution
Since there is a poor fit, take the log and try again

In [ ]:
# Plot sale prices against Normal Distribution
df_target = df_train.SalePrice
sns.distplot(df_target, fit=norm)
(mu, sigma) = norm.fit(df_target)
print('mu = {:2f} and sigma = {:2f}'.format(mu,sigma))
plt.legend(['Normal Districution'], loc='best')
plt.ylabel('Frequency')
plt.title('Sale Price Distribution')
fig = plt.figure()
res = stats.probplot(df_target, plot=plt)
plt.show()

In [ ]:
# Log transform target
df_target = np.log1p(df_target) #log of price + 1
sns.distplot(df_target, fit=norm)
(mu, sigma) = norm.fit(df_target)
print('mu = {:2f} and sigma = {:2f}'.format(mu,sigma))
plt.legend(['Normal Districution'], loc='best')
plt.ylabel('Frequency')
plt.title('Sale Price Distribution')
fig = plt.figure()
res = stats.probplot(df_target, plot=plt)
plt.show()

# Numerical Features
Check for skew
Take log

In [ ]:
# Check for skew and transforms those with high skew
skew_cutoff = 0.5
skew_values = num_data.apply(lambda x: skew(x))
skew_values = skew_values[abs(skew_values)>skew_cutoff]
print(skew_values.sort_values(ascending=False))
skew_features = skew_values.index
num_data[skew_features] = np.log1p(num_data[skew_features])

In [ ]:
# Normalize values
stdSc = StandardScaler()
num_data.loc[:,num_features] = stdSc.fit_transform(num_data.loc[:,num_features])

In [ ]:
# Plot distribution of transformed and normalized numerical features
f=pd.melt(num_data, value_vars = num_features)
g = sns.FacetGrid(f, col='variable', col_wrap=3, sharex=False, sharey=False)
g=g.map(sns.distplot, "value")

# Categorical Features

In [ ]:
# Plot Cateogrical Features
for each in cat_features:
    cat_data[each] = cat_data[each].astype('category')
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x,y=y)
    x=plt.xticks(rotation=90)
f=pd.melt(df_train, id_vars=['SalePrice'], value_vars = cat_features)
g = sns.FacetGrid(f, col='variable', col_wrap=2, sharex=False, sharey=False, size=5)
g=g.map(boxplot, "value", "SalePrice")


In [ ]:
# Test for differences between feature categories (ANOVA)
anova = pd.DataFrame()
anova['Feature'] = cat_features
values = []
for feature in cat_features:
    prices = []
    for cat in df_train[feature].unique():
        s = df_train[df_train[feature]==cat]['SalePrice'].values
        prices.append(s)
    pval = stats.f_oneway(*prices)[1]
    values.append(pval)
anova['Values'] = values
anova.sort_values('Values', inplace=True)
anova['Disparity'] = np.log(1./anova['Values'].values)

# Plot disparity by category
plt.figure(figsize=(10,5))
sns.barplot(data=anova, x='Feature', y='Disparity')
x=plt.xticks(rotation=90)

In [ ]:
# Create Dummy Variables for cateories
cat_data = pd.get_dummies(cat_data)

In [ ]:
# Data Frame Sizes
print('Data Frame Sizes after expanding categorical features:')
print('Num Data: ', num_data.shape)
print('Cat Data: ', cat_data.shape)
all_data = pd.concat([num_data, cat_data], axis=1, ignore_index=False)
print('All Data: ', all_data.shape)

# Identify uncoorelated Features to remove
Reduce number of features to avoid overfitting

In [ ]:
# use spearman coorlation to find features most coorelated with saleprice
spearman = pd.DataFrame()
features = num_features + cat_data.columns.tolist()
new_df = all_data[0:df_target.shape[0]]
spearman['Feature'] = features
spearman['Spearman'] = [new_df[f].corr(df_train['SalePrice'], 'spearman') for f in features]
spearman = spearman.sort_values('Spearman')
plt.figure(figsize=(6, 0.25*len(features)))
sns.barplot(data=spearman, y='Feature', x='Spearman', orient='h')

In [ ]:
# refine features to those most coorelated to sale price
# cutoff value chosen by comparing model error (below) for different values
spearman_cutoff = 0.1
refined_features = spearman[np.abs(spearman['Spearman'])>spearman_cutoff].Feature.tolist()
ref_df = all_data[refined_features]

In [ ]:
# Final dataframes
train = ref_df[0:ntrain]
test = ref_df[ntrain:]
target = df_target

# Print sizes of dataframes for modeling
print('Data Frame Sizes:')
print('Training: ', train.shape)
print('Target: ' , target.shape)
print('Testing: ' , test.shape)

# Modeling
linear regression

In [ ]:
# Divide training data for cross validation
test_size = 0.3
X_train, X_val, y_train, y_val = train_test_split(train, target,test_size = test_size, random_state=0)
print('Data divided for cross validation')
print('X_train: ', X_train.shape[0])
print('X_val: ', X_val.shape[0])
print('y_train: ', y_train.shape[0])
print('y_val: ', y_val.shape[0])

In [ ]:
# Define scoring with cross validation
n_folds = 5
def rmse_train(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = n_folds))
    return(rmse)
def rmse_val(model):
    rmse= np.sqrt(-cross_val_score(model, X_val, y_val, scoring="neg_mean_squared_error", cv = n_folds))
    return(rmse)

In [ ]:
#  Keep track of model validation errors
models = {}

In [ ]:
# Linear Regression without Regularization
lr = LinearRegression()
lr.fit(X_train, y_train)
print ('Linear Regression')
print('Training Error: ', rmse_train(lr).mean())
print('Validation Error: ', rmse_val(lr).mean())
models['LinearRegression'] = rmse_val(lr).mean()

In [ ]:
# Ridge Regression
print('Ridge CV')

# Find alpha
ridge = RidgeCV(alphas = [0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6, 10, 30, 60])
ridge.fit(X_train, y_train)
alpha = ridge.alpha_
print("Best alpha :", alpha)

print("Try again for more precision with alphas centered around " + str(alpha))
ridge = RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], 
                cv = 10)
ridge.fit(X_train, y_train)
alpha = ridge.alpha_
print("Best alpha :", alpha)

# Eliminated features
coefs = pd.Series(ridge.coef_, index = X_train.columns)
print("Ridge picked " + str(sum(coefs != 0)) + " features and eliminated the other " +  \
      str(sum(coefs == 0)) + " features")

# Get model error
print('Training Error: ', rmse_train(ridge).mean())
print('Validation Error: ', rmse_val(ridge).mean())
models['RidgeCV'] = rmse_val(ridge).mean()

In [ ]:
# Lasso Regression
print('Lasso CV')

# Find alpha
lasso = LassoCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1,0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
lasso.fit(X_train, y_train)
alpha = lasso.alpha_
print("Best alpha :", alpha)

print("Try again for more precision with alphas centered around " + str(alpha))
lasso = LassoCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], 
                max_iter = 50000, cv = 10)
lasso.fit(X_train, y_train)
alpha = lasso.alpha_
print("Best alpha :", alpha)

# Eliminated features
coefs = pd.Series(lasso.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coefs != 0)) + " features and eliminated the other " +  \
      str(sum(coefs == 0)) + " features")

# Get model error
print('Training Error: ', rmse_train(lasso).mean())
print('Validation Error: ', rmse_val(lasso).mean())
models['LassoCV'] = rmse_val(lasso).mean()

In [ ]:
models

# Run Model on Test Data

In [ ]:
test['SalePrice'] = np.expm1(lasso.predict(test))
test['Id'] = test_IDs

In [ ]:
# Final submission file
df_submit = test[['Id', 'SalePrice']]
df_submit.to_csv("../working/ridgeCV.csv", index=False)

In [ ]:
df_submit.head()